In [19]:
from simulation import *
from aesthetics import *
from models import *
from data import *
from analysis import *

import glob
import os as os
import re as re
import pandas as pd
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2
%matplotlib inline
mpl.rc('text', usetex=True)
cmap = sns.color_palette("Paired", 10)

# Question: do the angles in ADK look like HIVP when the catalytic rate is lowered?

In [27]:
adk_dir = '../../md-data/adenylate-kinase'
adk_unbound_files = sorted(glob.glob(adk_dir + '/AdKDihedHist_apo-4ake/' + '*'))
names = []
for file in range(len(adk_unbound_files)):
    name = os.path.splitext(os.path.basename(adk_unbound_files[file]))[0]
    name = re.search('^[^_]*', name).group(0)        
    if re.search('omega*', name):
        continue
    if re.search('chi3ASN*', name):
        continue
    if re.search('chi5LYS*', name):
        continue
    names.append(name)

In [12]:
def summarize_fluxes(name, concentration, data_source='adk_md_data', catalytic_rate=None):
    this = simulation(data_source = data_source)
    this.cSubstrate = concentration
    if catalytic_rate:
        this.catalytic_rate = catalytic_rate
    this.name = name
    this.simulate()
    directional_flux = np.mean(this.flux_u + this.flux_b)
    intersurface_flux = max(abs(this.flux_ub))
    # Make all flux on each surface positive
    unbound_flux = abs(this.flux_u)
    bound_flux = abs(this.flux_b)
    # Now find the maximum on either surface
    max_unbound = max(unbound_flux)
    max_bound = max(bound_flux)
    driven_flux = max([max_unbound, max_bound])
    del this
    return directional_flux, intersurface_flux, driven_flux


In [13]:
def summarize_power_and_load(name, concentration, data_source='adk_md_data', negative=False,
                            debug=False, catalytic_rate=None):
    this = simulation(data_source=data_source)
    this.cSubstrate = concentration
    if catalytic_rate:
        this.catalytic_rate = catalytic_rate
    this.name = name
    # The difference now, is that we're going to need to simulate with
    # an applied load.
    this.load = True
    # Initialize the applied load to be something small, and we'll increase
    # from there on out.
    slope = 0.000
    increment = 0.00001
    loads = []
    power_given_load = []
    flux_given_load = []
    while True:
        if negative:
            this.load_slope = -slope
        else:
            this.load_slope = slope
        this.simulate()
        # Bookkeeping
        flux = np.mean(this.flux_u + this.flux_b)
        power = this.load_slope * np.mean(this.flux_u + this.flux_b)
        loads.append(this.load_slope)
        flux_given_load.append(flux)
        power_given_load.append(power)
        # Now, we need to check the power trend to see if we should break
        # out of the loop.
        max_power = max(power_given_load)
        max_power_index = power_given_load.index(max_power)
        # Is the maximum power the last element?
        if max_power > power_given_load[0] and max_power > power_given_load[-1]:
            # print('Looks good. Maximum power = {} with load = {}'.format(
            #    max_power, loads[max_power_index]))
            # But let's check and make sure the power doesn't continue to increase,
            # before we break
            slope += increment
            if negative:
                this.load_slope = -slope
            else:
                this.load_slope = slope
            this.simulate()
            power = this.load_slope * np.mean(this.flux_u + this.flux_b)
            if max_power > power:
                # break
                return max_power, loads[max_power_index]
        if max_power == 0 and len(power_given_load) > 100:
            # Sometimes this happens... 
            # We should probably make a note of this too. Ugh.
            # This can happen if the maximum load that can be supported is even 
            # smaller than the increment size.
            return 0.0, 0.0
        if len(power_given_load) % 100 == 0:
            increment *= 10
        if len(power_given_load) > 1000:
            print('I give up.')
        if slope > 10:
            print('This doesn\'t make sense.')
            break
        slope += increment
    del this

In [29]:
calculation = True
df = pd.DataFrame()
concentrations = np.arange(-6, 0, 0.1)
if calculation:
    for concentration in tqdm(concentrations):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                               data_source='adk_md_data',
                                                                               catalytic_rate=10) 
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('adk-low-catalytic-rate-10.pickle')

  0%|          | 0/60 [00:00<?, ?it/s]

concentration = -6.0


  2%|▏         | 1/60 [00:15<15:19, 15.58s/it]

concentration = -5.9


  3%|▎         | 2/60 [00:30<14:50, 15.35s/it]

...

 97%|█████████▋| 58/60 [15:19<00:36, 18.44s/it]

concentration = -0.2000000000000206


 98%|█████████▊| 59/60 [15:38<00:18, 18.54s/it]

concentration = -0.10000000000002096


100%|██████████| 60/60 [15:56<00:00, 18.54s/it]
